In [ ]:
# All the imports go here
import cv2
import numpy as np
from collections import deque

# Import specific modules
from mediapipe.python.solutions import hands as mp_hands
from mediapipe.python.solutions import drawing_utils as mp_drawing
def save_canvas(canvas, filename):
    cv2.imwrite(filename, canvas)
# Giving different arrays to handle colour points of different colour
bpoints = [deque(maxlen=1024)]
gpoints = [deque(maxlen=1024)]
rpoints = [deque(maxlen=1024)]
ypoints = [deque(maxlen=1024)]

# These indexes will be used to mark the points in particular arrays of specific colour
blue_index = 0
green_index = 0
red_index = 0
yellow_index = 0

# The kernel to be used for dilation purpose 
kernel = np.ones((5,5),np.uint8)

colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 255, 255)]
colorIndex = 0

# Here is code for Canvas setup
paintWindow = np.zeros((471,636,3)) + 255
paintWindow = cv2.rectangle(paintWindow, (40,1), (140,65), (0,0,0), 2)
paintWindow = cv2.rectangle(paintWindow, (160,1), (255,65), (255,0,0), -1)
paintWindow = cv2.rectangle(paintWindow, (275,1), (370,65), (0,255,0), -1)
paintWindow = cv2.rectangle(paintWindow, (390,1), (485,65), (0,0,255), -1)
paintWindow = cv2.rectangle(paintWindow, (505,1), (600,65), (0,255,255), -1)

cv2.putText(paintWindow, "CLEAR", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

cv2.namedWindow('Paint', cv2.WINDOW_AUTOSIZE)

# initialize mediapipe
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp_drawing

# Initialize the webcam
cap = cv2.VideoCapture(0)
ret = True
while ret:
    # Read each frame from the webcam
    ret, frame = cap.read()

    x, y, c = frame.shape

    # Flip the frame vertically
    frame = cv2.flip(frame, 1)
    #hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    frame = cv2.rectangle(frame, (40,1), (140,65), (0,0,0), 2)
    frame = cv2.rectangle(frame, (160,1), (255,65), (255,0,0), -1)
    frame = cv2.rectangle(frame, (275,1), (370,65), (0,255,0), -1)
    frame = cv2.rectangle(frame, (390,1), (485,65), (0,0,255), -1)
    frame = cv2.rectangle(frame, (505,1), (600,65), (0,255,255), -1)
    cv2.putText(frame, "CLEAR", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    #frame = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    # Get hand landmark prediction
    result = hands.process(framergb)

    # post process the result
    if result.multi_hand_landmarks:
        landmarks = []
        for handslms in result.multi_hand_landmarks:
            for lm in handslms.landmark:
                # # print(id, lm)
                # print(lm.x)
                # print(lm.y)
                lmx = int(lm.x * 640)
                lmy = int(lm.y * 480)

                landmarks.append([lmx, lmy])

            # Drawing landmarks on frames
            mpDraw.draw_landmarks(frame, handslms, mp_hands.HAND_CONNECTIONS)
        fore_finger = (landmarks[8][0],landmarks[8][1])
        center = fore_finger
        thumb = (landmarks[4][0],landmarks[4][1])
        cv2.circle(frame, center, 3, (0,255,0),-1)
        print(center[1]-thumb[1])
        if (thumb[1]-center[1]<30):
            bpoints.append(deque(maxlen=512))
            blue_index += 1
            gpoints.append(deque(maxlen=512))
            green_index += 1
            rpoints.append(deque(maxlen=512))
            red_index += 1
            ypoints.append(deque(maxlen=512))
            yellow_index += 1

        elif center[1] <= 65:
            if 40 <= center[0] <= 140: # Clear Button
                bpoints = [deque(maxlen=512)]
                gpoints = [deque(maxlen=512)]
                rpoints = [deque(maxlen=512)]
                ypoints = [deque(maxlen=512)]

                blue_index = 0
                green_index = 0
                red_index = 0
                yellow_index = 0

                paintWindow[67:,:,:] = 255
            elif 160 <= center[0] <= 255:
                    colorIndex = 0 # Blue
            elif 275 <= center[0] <= 370:
                    colorIndex = 1 # Green
            elif 390 <= center[0] <= 485:
                    colorIndex = 2 # Red
            elif 505 <= center[0] <= 600:
                    colorIndex = 3 # Yellow
        else :
            if colorIndex == 0:
                bpoints[blue_index].appendleft(center)
            elif colorIndex == 1:
                gpoints[green_index].appendleft(center)
            elif colorIndex == 2:
                rpoints[red_index].appendleft(center)
            elif colorIndex == 3:
                ypoints[yellow_index].appendleft(center)
    # Append the next deques when nothing is detected to avoid messing up
    else:
        bpoints.append(deque(maxlen=512))
        blue_index += 1
        gpoints.append(deque(maxlen=512))
        green_index += 1
        rpoints.append(deque(maxlen=512))
        red_index += 1
        ypoints.append(deque(maxlen=512))
        yellow_index += 1

    # Draw lines of all the colors on the canvas and frame
    points = [bpoints, gpoints, rpoints, ypoints]
    for i in range(len(points)):
        for j in range(len(points[i])):
            for k in range(1, len(points[i][j])):
                if points[i][j][k - 1] is None or points[i][j][k] is None:
                    continue
                cv2.line(frame, points[i][j][k - 1], points[i][j][k], colors[i], 2)
                cv2.line(paintWindow, points[i][j][k - 1], points[i][j][k], colors[i], 2)

    cv2.imshow("Output", frame) 
    cv2.imshow("Paint", paintWindow)
    # Check if the 's' key is pressed to save the canvas
    key = cv2.waitKey(1)
    if key == ord('s'):
        # Generate a unique filename based on the current timestamp
        filename = f"canvas_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.png"
        save_canvas(paintWindow, filename)
        print(f"Canvas saved as {filename}")

    # Check if the 'Esc' key is pressed to exit the application
    elif key == 27:
        break

cap.release()
cv2.destroyAllWindows()


D:\anaconda\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


-113
-127
-126
-127
-131
-131
-131
-131
-116
-119
-121
-120
-106
-107
-108
-118
-120
-119
-119
-126
-128
-128
-127
-126
-126
-126
-117
-121
-121
-120
-121
-120
-121
-121
-118
-116
-117
-115
-114
-116
-115
-120
-120
-121
-121
-122
-123
-123
-123
-115
-116
-116
-116
-119
-121
-122
-123
-124
-124
-124
-120
-123
-123
-123
-122
-120
-121
-121
-117
-117
-117
-117
-113
-113
-113
-111
-112
-112
-112
-111
-109
-110
-109
-105
-107
-107
-106
-106
-105
-105
-98
-96
-96
-96
-97
-95
-94
-100
-100
-99
-99
-96
-97
-97
-97
-95
-95
-95
-95
-93
-91
-91
-84
-84
-84
-84
-86
-85
-84
-85
-79
-78
-78
-77
-71
-70
-67
-67
-69
-72
-71
-66
-67
-67
-67
-62
-55
-50
-57
-60
-61
-62
-61
-57
-50
-43
-36
-37
-35
-35
-30
-37
-46
-43
-49
-48
-49
-49
-48
-49
-48
-41
-45
-44
-44
-41
-41
-43
-37
-37
-36
-38
-36
-38
-38
-37
-40
-42
-40
-33
-38
-38
-38
-38
-36
-35
-35
-30
-29
-29
-29
-23
-25
-23
-24
-69
-64
-65
-66
-107
-108
-108
-108
-105
-103
-104
-106
-106
-106
-106
-102
-102
-102
-102
-100
-101
-101
-101
-98
-98
-98
-94
-

-16
-9
-10
-20
-21
-20
-18
-74
-76
-77
-76
-94
-92
-93
-93
-101
-100
-100
-100
-102
-103
-102
-105
-103
-104
-102
-92
-93
-94
-93
-70
-62
-65
-64
-10
-13
-13
-14
-15
-15
-14
-13
-13
-13
-13
-11
-12
-12
-11
-8
-7
-8
-6
-9
-10
-10
-6
-8
-8
-9
-11
-11
-11
-25
-28
-31
-12
-16
-16
-17
-16
-44
-41
-40
-41
-73
-77
-76
-79
-81
-81
-81
-85
-86
-86
-85
-92
-94
-94
-94
-96
-96
-96
-96
-97
-98
-97
-95
-97
-97
-97
-97
-97
-96
-96
-98
-99
-98
-98
-98
-99
-99
-97
-98
-99
-99
-93
-95
-95
-95
-93
-93
-92
-98
-98
-98
-98
-104
-102
-101
-101
-108
-105
-106
-106
-105
-105
-105
-101
-102
-102
-103
-102
-102
-102
-102
-102
-102
-102
-102
-106
-104
-105
-106
-104
-106
-105
-108
-108
-107
-107
-109
-106
-106
-107
-107
-105
-105
-105
-109
-108
-108
-113
-113
-114
-114
-109
-107
-108
-108
-104
-104
-105
-105
-103
-103
-104
-103
-103
-103
-103
-104
-105
-105
-105
-103
-104
-103
-104
-103
-105
-104
-104
-103
-103
-103
-102
-103
-103
-103
-103
-103
-103
-104
-104
-104
-103
-100
-98
-99
-99
-92
-97
-95
-96
-11
-15


-134
-136
-136
-64
-86
-101
-98
-60
-58
-83
-98
-129
-129
-127
-127
-132
-135
-134
-135
-149
-148
-148
-148
-110
-111
-109
-63
-112
-127
-127
-136
-137
-139
-137
-129
-130
-133
-134
-88
-106
-101
-139
-140
-141
-141
-142
-146
-146
-145
-163
-171
-172
-160
-154
-158
-155
-161
-159
-160
-161
-145
-152
-150
-170
-174
-175
-173
-152
-154
-152
-153
-151
-156
-154
-154
-186
-196
-198
-198
-137
-139
-137
-150
-154
-148
-148
-158
-157
-155
-167
-159
-163
-161
-154
-145
-143
-144
-137
-137
-138
-138
-140
-138
-139
-133
-134
-133
-133
-124
-128
-128
-129
-120
-148
-155
-155
-151
-160
-162
-165
-152
-150
-153
-160
-91
-82
-86
-92
-140
-148
-144
-145
-71
-83
-106
-98
-94
-100
-100
-149
-180
-181
-187
-187
-187
-186
-190
-186
-187
-177
-176
-177
-176
-164
-161
-162
-163
-156
-158
-158
-159
-164
-166
-166
-179
-177
-177
-177
-180
-180
-176
-178
-99
-18
-45
-80
-30
-60
-32
-46
-100
-134
-117
-155
-171
-185
-182
-173
-171
-171
-171
-183
-186
-186
-185
-175
-178
-178
-177
-171
-170
-171
-181
-188
-183
